In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\asociacion_sigfe_sigcom.xlsx')
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'].str.replace('.', '', regex = False)
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.set_index('Cod. SIGFE limpio')

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

In [22]:
class AnalizadorSIGCOM:
    def __init__(self):
        pass
    
    def correr_programa(self):
        estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom = self.cargar_archivos_y_tratar_df()
        suma_gg_por_sigfe_total = self.obtener_suma_gg_total_por_sigfe(estado_ej_presup)
        suma_gg_por_sigfe_sin_rrhh, suma_rrhh = self.obtener_gastos_gg_y_rrhh(suma_gg_por_sigfe_total, disponibilidad_devengo)

        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.groupby('Cod. SIGCOM', dropna = False).sum()
        formato_rellenado = self.rellenar_formato(suma_gg_por_sigcom_sin_rrhh, formato_gg_sigcom)

        self.guardar_archivos(suma_gg_por_sigcom_sin_rrhh, suma_rrhh, formato_rellenado)

    def cargar_archivos_y_tratar_df(self):
        estado_ej_presup = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
        estado_ej_presup['Item Presupuestario'] = estado_ej_presup['Concepto Presupuestario'].str.split().str[0]
        estado_ej_presup = estado_ej_presup.set_index('Item Presupuestario')
    
        disponibilidad_devengo = df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
        disponibilidad_devengo['Item Presupuestario'] = disponibilidad_devengo['Concepto Presupuestario'].str.split().str[0]
        disponibilidad_devengo = disponibilidad_devengo[['Titulo', 'Principal', 'Concepto Presupuestario', 'Item Presupuestario', 'Monto Vigente', 'Monto Disponible', 'Monto Consumido']]
        
        formato_gg_sigcom = pd.read_excel('input\\Formato 3_Gasto General 2022-09_2.xlsx')
        formato_gg_sigcom = formato_gg_sigcom.set_index('Unnamed: 0')

        return estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom

    
    def obtener_suma_gg_total_por_sigfe(self, estado_ej_presup):
        unidas = pd.merge(estado_ej_presup, TRADUCTOR_SIGFE_SIGCOM, how = 'left', left_index = True, right_index = True)
    
        agrupados_por_sigfe = unidas.groupby(['Cod. SIGCOM', 'Item en SIGCOM', 'Item Presupuestario'], dropna = False)['Devengado'].sum().reset_index()
        agrupados_por_sigfe['Devengado_GG_consolidado'] = agrupados_por_sigfe['Devengado']

        agrupados_por_sigfe = agrupados_por_sigfe.rename(columns = {'Devengado': 'Devengado_EstadoEjecucionPresupuestaria',
                                                                    'Devengado_RRHH': 'Devengado_RRHH_DisponibilidadDevengoPresupuestario',
                                                                    'Devengado_GG': 'Devengado_GG_DisponibilidadDevengoPresupuestario'})

        return agrupados_por_sigfe

    
    def obtener_gastos_gg_y_rrhh(self, suma_gg_por_sigfe, disponibilidad_devengo):
        df_rrhh_total = pd.DataFrame()
        suma_gg_por_sigfe_sin_rrhh = suma_gg_por_sigfe.copy()

        for concepto_pres_excepcion in EXCEPCIONES_SIGFE.keys():
            query_excepcion = disponibilidad_devengo.query('`Item Presupuestario` == @concepto_pres_excepcion')

            if concepto_pres_excepcion == '221299901601':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
            
            elif concepto_pres_excepcion == '221299901602':
                mask_a_rrhh = query_excepcion['Principal'].notna()
            
            elif concepto_pres_excepcion == '221299900902':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('CARDIOLOGIA') | (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

            elif concepto_pres_excepcion == '221299900201':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
                
            elif concepto_pres_excepcion == '221299900202':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
            
            df_a_rrhh = query_excepcion[mask_a_rrhh]
            df_rrhh_total = pd.concat([df_rrhh_total, df_a_rrhh])

            df_a_gg = query_excepcion[~mask_a_rrhh]

            valor_a_rrhh = df_a_rrhh['Monto Vigente'].sum()
            valor_a_gg = df_a_gg['Monto Vigente'].sum()

            mask_excepcion = (suma_gg_por_sigfe_sin_rrhh['Item Presupuestario'] == concepto_pres_excepcion)

            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_RRHH'] = valor_a_rrhh 
            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_GG'] = valor_a_gg
            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_GG_consolidado'] = valor_a_gg

        return suma_gg_por_sigfe_sin_rrhh, df_rrhh_total
    
    def rellenar_formato(self, suma_gg_por_sigcom_sin_rrhh, formato_sigcom_gg):
        columnas_antiguas = formato_sigcom_gg.columns
        formato_sigcom_gg.columns = formato_sigcom_gg.columns.str.split('-').str[0].astype(float)

        for codigo_gasto in suma_gg_por_sigcom_sin_rrhh.index:
            if codigo_gasto in formato_sigcom_gg.columns:
                valor_a_ingresar = suma_gg_por_sigcom_sin_rrhh.loc[codigo_gasto, 'Devengado_GG_consolidado']
                formato_sigcom_gg.loc['Valor General', codigo_gasto] = valor_a_ingresar

        formato_sigcom_gg.columns = columnas_antiguas
        formato_sigcom_gg = formato_sigcom_gg.reset_index()
        
        return formato_sigcom_gg
    
    def guardar_archivos(self, suma_gg_por_sigcom_sin_rrhh, suma_rrhh, formato_rellenado):
        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigcom_sin_rrhh.reset_index()
        suma_rrhh = suma_rrhh.reset_index()
        formato_rellenado = formato_rellenado.reset_index()

        with pd.ExcelWriter('output.xlsx') as writer:
            formato_rellenado.to_excel(writer, sheet_name = 'formato_listo', index = False)
            suma_gg_por_sigcom_sin_rrhh.to_excel(writer, sheet_name = 'suma_gg_sin_rrhh', index = False)
            suma_rrhh.to_excel(writer, sheet_name = 'suma_rrhh', index = False)


In [23]:
objeto = AnalizadorSIGCOM()
objeto.correr_programa()